In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import sys
sys.path.append("../../..")
import numpy as np
import pandas as pd
import geopandas as gpd
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from pyproj import Proj, transform

In [ ]:
# Creating the network for Brussels. 
# fill in these parameters 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
ext = 0  # 0 or 2
buffer = 3
city = 'BRUSSEL'

network = road_network_from_place("Brussels", buffer_dist_close=buffer*1000)
network = relabel_graph(network)
#show_network(network,notebook=True)

In [ ]:
# string checks
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'

zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
od_path2 = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
od_path0 = data_path + os.path.sep + "od_matrix_data" + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"

print('zoning path: ' + zoning_path)
print('od path (if ext = 2): ' + od_path2)
print('od path (if ext = 0): ' + od_path0)

In [ ]:
# Read in the zoning file
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'
zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
zoning = gpd.read_file(zoning_path) # Everything is now stored in a geopandas dataframe. 

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
connector_type = 'link'
network = add_centroids(network, x_centroids,y_centroids,k=1, method=connector_type)
network = relabel_graph(network)
show_network(network, notebook=True)

In [ ]:
od = pd.read_excel(od_path2)
old_od = od.to_numpy() # The OD matrix is now stored in a numpy array

# Create OD graph and plot demand
old_graph = od_graph_from_matrix(old_od,x_centroids,y_centroids) 
# demand plot is useless since there are too many OD-pairs 

In [ ]:
# A matrix
A = 1.5
# B matrix
B = 0.05

In [ ]:
# First iteration
assignment = StaticAssignment(network,old_graph)
result = assignment.run('dial_b')    
new_od = (A-result.skim)/B
print(np.max(abs((new_od-old_od))))

In [ ]:
i = 1 
max_iterations = 3
max_deviation_per_od = 1
while np.max(abs((new_od - old_od))) > max_deviation_per_od and i < max_iterations:
    # Perhaps MSA stap
    old_od = new_od
    old_graph =  od_graph_from_matrix(old_od, x_centroids, y_centroids)
    assignment = StaticAssignment(network,old_graph)
    result = assignment.run('dial_b')
    new_od = (A-result.skim)/B
    i += 1
    print(np.max(abs((new_od-old_od))))